In [3]:
import json
import csv

# Specify the input and output file paths
input_file = 'NIKL_AU_2023_COMPETITION_v1.0/nikluge-au-2022-test.jsonl'
output_file = 'NIKL_AU_2023_COMPETITION_v1.0/nikluge-au-2022-test.csv'

data = []

# Read the JSONL file
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        # Remove any leading/trailing whitespace
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        # Parse the JSON object
        record = json.loads(line)
        data.append(record)

# Write data to a CSV file
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    # Define the CSV column headers
    fieldnames = ['id', 'input', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Write each JSON object as a row in the CSV file
    for record in data:
        writer.writerow(record)


In [4]:
# 데이터 읽어와보기
import pandas as pd
train_df = pd.read_csv("NIKL_AU_2023_COMPETITION_v1.0/nikluge-au-2022-train.csv")

In [21]:
len(train_df[train_df.output==1])

6786

In [22]:
len(train_df[train_df.output==0])

9794

In [26]:
for i, j in train_df.items():
    print(i)
    print("0"*100)
    print(j)

id
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0        nikluge-au-2022-train-000001
1        nikluge-au-2022-train-000002
2        nikluge-au-2022-train-000003
3        nikluge-au-2022-train-000004
4        nikluge-au-2022-train-000005
                     ...             
16575    nikluge-au-2022-train-016576
16576    nikluge-au-2022-train-016577
16577    nikluge-au-2022-train-016578
16578    nikluge-au-2022-train-016579
16579    nikluge-au-2022-train-016580
Name: id, Length: 16580, dtype: object
input
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0        보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사...
1        왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프...
2        이런 쓰레기같은 새끼가 아무렇지 않게 멀쩡히 돌아다닐 생각을 하니까 진짜 너무 소름...
3                                             인간의 탈을 쓰고...
4                              인기글에 짱깨뭐라하니까 댓글로 ㅂㄷㅂㄷ하네요...
                               ...   

In [31]:
from transformers import AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

/Users/jo_nyuk/.pyenv/versions/3.11.8/envs/LangChain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
# tokenizer test
tokenizer(train_df['input'].tolist())

{'input_ids': [[2, 3897, 2223, 31369, 1460, 12814, 2529, 2088, 5508, 1889, 2259, 2728, 2119, 12494, 3683, 18, 12908, 1160, 31369, 1460, 3897, 2223, 2529, 2088, 1889, 2259, 3611, 3893, 17270, 2062, 18, 18, 3], [2, 1460, 3907, 12443, 2069, 3861, 6233, 3897, 2223, 2088, 1335, 2073, 2118, 28044, 1378, 4314, 2154, 10, 79, 6624, 5148, 10, 1326, 3701, 2208, 2228, 2241, 4177, 1517, 2200, 1, 880, 6261, 693, 2088, 10817, 8071, 2088, 717, 2379, 4273, 2097, 2776, 2315, 35, 543, 5412, 28674, 18, 3], [2, 3667, 6614, 2246, 2073, 7171, 2116, 12620, 2118, 1380, 2318, 14430, 2468, 3812, 24498, 3628, 2069, 1889, 3707, 4229, 3760, 16109, 3151, 2259, 2062, 18, 3], [2, 3804, 2079, 1764, 2069, 1363, 2088, 18, 18, 18, 3], [2, 4316, 2701, 2170, 1602, 2844, 3005, 2181, 13679, 2299, 7757, 2200, 187, 2553, 2243, 2553, 2205, 2203, 2182, 18, 18, 18, 3], [2, 3851, 11857, 2440, 4142, 31369, 1460, 3914, 3628, 2205, 18246, 1097, 636, 3311, 2180, 6321, 2088, 1, 3], [2, 6042, 2170, 24693, 2907, 2051, 3], [2, 4785, 1891, 